In [20]:
# Imports -  These are all you need for the assignment: do not import additional packages
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
from geopy.geocoders import Nominatim
from urllib.request import Request

from pandas.core import datetools
import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest


In [34]:
#load Los Angeles Data
df_LosAngeles = pd.read_csv("data/Crime_Data_from_2010_to_Present.csv") 

#Remove unneeded columns
df_LosAngeles_Trimmed = df_LosAngeles[['Date Occurred','Time Occurred','Crime Code','Location ']]

In [22]:
#Load Median Housing Data
df_HousePrice = pd.read_csv("data/Zip_MedianListingPrice_AllHomes.csv")

#Filter by Los Angeles Only
df_LAHousePrice = df_HousePrice[df_HousePrice['City'] == 'Los Angeles']

In [74]:
#Load US Zip Codes
df_US_Zip = pd.read_csv("data/US.txt",header = None, sep = '\t')
df_US_Zip.columns = ['Country', 'ZipCode','City','State','Stabv','ignore1','ignore2','ignore3', 'ignore4','long','lat','ignore5']

#Sort Only Los Angeles
df_US_Zip_trimmed = df_US_Zip[df_US_Zip['City'] == 'Los Angeles']

df_US_Zip_trimmed = df_US_Zip_trimmed[['Country','ZipCode','City','State','long','lat']]

df_US_Zip_trimmed.head()

,Country,ZipCode,City,State,long,lat
2811,US,90001,Los Angeles,California,33.9731,-118.2479
2812,US,90002,Los Angeles,California,33.9497,-118.2462
2813,US,90003,Los Angeles,California,33.9653,-118.2727
2814,US,90004,Los Angeles,California,34.0762,-118.3029
2815,US,90005,Los Angeles,California,34.0585,-118.3012


In [ ]:
#Function to get closest zip code
def get_closest_zip(long, lat):

In [24]:
#locator hack

def get_geolocator():
    geolocator = Nominatim()

    requester = geolocator.urlopen

    def requester_hack(req, **kwargs):
        req = Request(url=req, headers=geolocator.headers)
        return requester(req, **kwargs)

    geolocator.urlopen = requester_hack

    return geolocator

In [54]:
#Convert location to address
location = get_geolocator().reverse('34.0454, -118.3157')
address = location.address
print(location)


1429, South Van Ness Avenue, Country Club Park, Cienega, LA, Los Angeles County, California, 90019, United States of America


In [46]:
df_LosAngeles_Trimmed.head()
df_LosAngeles.shape

(1692056, 26)

In [ ]:
df_HousePrice

In [36]:
#Sort los Angeles data by 
df_LosAngeles_Trimmed = df_LosAngeles_Trimmed.sort_values(by='Date Occurred')

In [37]:
df_LosAngeles_Trimmed['Location ']= df_LosAngeles_Trimmed['Location '].str.replace(r"\(","")
df_LosAngeles_Trimmed['Location ']= df_LosAngeles_Trimmed['Location '].str.replace(r"\)","")


In [67]:
df_LosAngeles_Trimmed.head(10)

,Date Occurred,Time Occurred,Crime Code,Location
254354,01/01/2010,1200,740,"34.1103, -118.3333"
319962,01/01/2010,2300,812,"33.795, -118.2711"
293441,01/01/2010,800,810,"33.9637, -118.3003"
319964,01/01/2010,1200,354,"33.7742, -118.262"
309604,01/01/2010,1,354,"34.2354, -118.4502"
319965,01/01/2010,600,354,"33.7569, -118.3026"
319966,01/01/2010,800,354,"33.7915, -118.2939"
319967,01/01/2010,1,354,"33.7951, -118.2752"
319969,01/01/2010,1,354,"33.7831, -118.2552"
138172,01/01/2010,1800,510,"34.2891, -118.4135"


In [73]:
df_LosAngeles_Trimmed_Location = df_LosAngeles_Trimmed['Location '].str.split(',', expand=True)
df_LosAngeles_Trimmed_Location.columns = ['Long', 'Lat']

df_LosAngeles_Trimmed_LongLat = pd.concat([df_LosAngeles_Trimmed,df_LosAngeles_Trimmed_Location],axis=1)

df_LosAngeles_Trimmed_LongLat

,Date Occurred,Time Occurred,Crime Code,Location,Long,Lat
254354,01/01/2010,1200,740,"34.1103, -118.3333",34.1103,-118.3333
319962,01/01/2010,2300,812,"33.795, -118.2711",33.795,-118.2711
293441,01/01/2010,800,810,"33.9637, -118.3003",33.9637,-118.3003
319964,01/01/2010,1200,354,"33.7742, -118.262",33.7742,-118.262
309604,01/01/2010,1,354,"34.2354, -118.4502",34.2354,-118.4502
319965,01/01/2010,600,354,"33.7569, -118.3026",33.7569,-118.3026
319966,01/01/2010,800,354,"33.7915, -118.2939",33.7915,-118.2939
319967,01/01/2010,1,354,"33.7951, -118.2752",33.7951,-118.2752
319969,01/01/2010,1,354,"33.7831, -118.2552",33.7831,-118.2552
138172,01/01/2010,1800,510,"34.2891, -118.4135",34.2891,-118.4135


In [76]:
#Use service to get zip, too much data doesn't work D:

#df_LosAngeles_Trimmed_test.assign(Address=df_LosAngeles_Trimmed_test['Location '].apply(get_geolocator().reverse))

df_LosAngeles_Trimmed_LongLat.to_csv('LosAngelesCrimeData.csv')

In [ ]:
#get zip code